In [1]:
% matplotlib inline
from __future__ import print_function

from collections import Counter
import json
import os
import codecs

import lda
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sparse
from sklearn.decomposition import LatentDirichletAllocation

<h2>データを読み込もう</h2>

In [2]:
with open(os.path.normpath('./dataset/document_word_data.json'), 'r') as f:
    doc_data = json.load(f)

all_doc_index = doc_data.keys()
print('Total Documents: ', len(all_doc_index))

Total Documents:  1000


Windowsの方でutf-8のテキストファイルが開けない場合は、こちらを実行してみてください。

In [3]:
# Windowsの方で上記のコードが失敗する場合
with codecs.open(os.path.normpath('./dataset/document_word_data.json'), 'r', "utf-8") as f:
    doc_data = json.load(f)

all_doc_index = doc_data.keys()
print('Total Documents: ', len(all_doc_index))

Total Documents:  1000


この読み込んだjsonデータはこんな感じです。

In [4]:
', '.join(doc_data['715'])

'ママ, 子供, 健康, づくり, 新た, ライフスタイル, 提案, ママ, マルシェ, 府, 府, 市, さまざま, 家族, ら, 日, もん, 商品, ほか, ハロ, 子供, 商品, プレゼント, 各日, 人, 会場, 木, ぬくもり, 木, 子供, づくり, 会, 木, 日, 午後, 時, 今年度, 森林, 林業, 木材, 大使, ミス, 日本, みどり, 帆, 南, さん, 府, 木材, 会, 湯川, 昌子, さん, ら, 女性, 人, 参加, スギ, ヒノキ, 木材, 放出, 健康, 効果, 木材, こと, 女性, 入場, 無料, 文化, 園, 入園, 料, 大人, 円, 円'

単語のインデックスを作るために、全ての単語のリストを作ります。

In [26]:
all_vocab = []
for doc_idx in all_doc_index:
    all_vocab += doc_data[doc_idx]

# 重複を消すためにsetしてlistにする
all_vocab = list(set(all_vocab))
vocab_num = len(all_vocab)
print('Vocablary Number: ', vocab_num)

Vocablary Number:  8709


In [28]:
all_doc_index_ar = np.array(list(all_doc_index))

train_portion = 0.7
train_num = int(len(all_doc_index_ar) * train_portion)

np.random.shuffle(all_doc_index_ar)
train_doc_index = all_doc_index_ar[:train_num]
test_doc_index = all_doc_index_ar[train_num:]

In [33]:
len(train_doc_index)
len(test_doc_index)

300

先にからっぽのスパース行列を定義します。

In [7]:
A_train = sparse.lil_matrix((len(train_doc_index), len(all_vocab)),
                            dtype=np.int)
A_test = sparse.lil_matrix((len(test_doc_index), len(all_vocab)),
                           dtype=np.int)

ListからNumpyのArrayに直します。

In [8]:
all_vocab_ar = np.array(all_vocab)
train_doc_index_ar = np.array(train_doc_index)
test_doc_index_ar = np.array(test_doc_index)

In [42]:
all_vocab_ar 

array(['有利', 'ぼうこう', '幽か', ..., 'ココ', 'ララビュウ', 'ステッチ'], dtype='<U18')

In [49]:
print(np.where(all_vocab_ar == 'ココ'))


(array([8706]),)


スパース行列に成分を入れていきます。

In [9]:
# 学習用
train_total_elements_num = 0
for i in range(len(train_doc_index)):
    doc_idx = train_doc_index[i]
    row_data = Counter(doc_data[doc_idx])
    
    for word in row_data.keys():
        word_idx = np.where(all_vocab_ar == word)[0][0]
        print(word_idx)
        A_train[i, word_idx] = row_data[word]
        train_total_elements_num += 1
print('Train total elements num :', train_total_elements_num)


# テスト用
test_total_elements_num = 0
for i in range(len(test_doc_index)):
    doc_idx = test_doc_index[i]
    row_data = Counter(doc_data[doc_idx])
    
    for word in row_data.keys():
        word_idx = np.where(all_vocab_ar == word)[0][0]
        A_test[i, word_idx] = row_data[word]
        test_total_elements_num += 1
print('Test total elements num :', test_total_elements_num)

Train total elements num : 33591
Test total elements num : 14453


###  CountVectorizerを用いてdoc_dataから簡単に疎行列を作ることもできます

In [52]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=lambda a: a, analyzer=lambda a: a)
vectorizer.fit(doc_data[idx] for idx in all_doc_index)
A_train = vectorizer.transform(doc_data[idx] for idx in train_doc_index)
A_test = vectorizer.transform(doc_data[idx] for idx in test_doc_index)

## 実際にLDAを適用してみよう (Scikit-learnを使った例）

In [53]:
model1 = LatentDirichletAllocation(n_topics=20,
                                   doc_topic_prior=0.001,
                                   topic_word_prior=0.5,
                                   max_iter=5,
                                   learning_method='online',
                                   learning_offset=50.,
                                   random_state=0)

In [12]:
model1.fit(A_train)

/Users/takahi-i/.pyenv/versions/3.6.0/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=0.001,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=20, perp_tol=0.1,
             random_state=0, topic_word_prior=0.5, total_samples=1000000.0,
             verbose=0)

まずトピック x 単語を見てみましょう

In [13]:
normalize_components = model1.components_ / model1.components_.sum(axis=0)

In [14]:
# http://scikit-learn.org/stable/auto_examples/applications/
# topics_extraction_with_nmf_lda.html　より
n_top_words = 20
for topic_idx, topic in enumerate(normalize_components):
    print('Topic #%d:' % topic_idx)
    print(' '.join([all_vocab_ar[i] for i in
                    topic.argsort()[:-n_top_words - 1:-1]]))
    print()

Topic #0:
メッキ とき カラオケアプリ 味噌 国営 スクエア 匂い ダメ グルテン 強 ウッチャン 志摩 心待ち 回 振り付け 本書 スポニチ さし パロアルト ガ

Topic #1:
スランプ 泥棒 ドデカイ 戦国 品 導師 ばら 浩 喜八 オタク 外国 マッサン ツヤ 枕 取りやめ ニオイ ワンポイント エッチ 入室 容

Topic #2:
エマナ 別人 切り出し 塁 ノイタミナ うちわ ぎゃるがん コ 実勢 協定 付け バルセロナキャンプ 女優 ランズ アダルト 他 気味 厚手 ウォッチ 手前

Topic #3:
ウィル 共和党 スタメン デスメタル 決済 ヘッドマウントディスプレイ 充分 ヴィブロス 大喜 来年度 合 キャビン オフショット 円 太もも セルフネイル ハロウィン シフト マンネリ 名声

Topic #4:
トゥットスポルト ビジョン レクサス 奥さん 戸塚 ウォッシュレット 交差点 出場 シンク ベネズエラ たいよう ダンス 凱旋 ロング ベイベ 将来 平塚 改憲 リッチ チカラ

Topic #5:
復帰 次元 島崎 アミュプラザ おじ 少年 日常 ブラウン にいさん 兆 尻 まちなか 国広 ガガ ドコモ 完全 向け ニキビ クロス 栗子

Topic #6:
京子 モロヘイヤ 点数 タロウ 台東 お悔やみ 栗山 ボンボン プレゼンス ダロワイヨ むや 担保 ヒユウザン ビフィズス ドリブル 武司 セ チャント クリントン てん

Topic #7:
樋口 就学 ランキング さ タブ いっしょ 本体 滑 まんま 毛 こまめ メカニズム コンビニ プラティニ 吉川 人 スタンプ ナゾ 温暖 ブロフェス

Topic #8:
検察 江北 ラメ 住処 コシ 沿岸 段数 ダウ ヒト 伝染 オブ 滅多 タオル ラッチ 将希 応援 サウンド 可 ブルックリン 決意

Topic #9:
前倒し 混入 検出 楽 可塑 ウォッシュレット 専業 時代 合体 凄惨 東屋 低俗 低め 月刊 オレンジ だらけ 同情 ジワッ パッティング 刃

Topic #10:
スジョン ツイン 意気投合 イブニングキャンプ キュン 技法 オビ 招き ヘクセイタス スポンジ ヒョウ エチケット 同日 振幅 点検 マイルド だし ハイレベル 展望 庭

Topi

文書 x トピック行列側も見てみましょう。

In [15]:
doc_topic_data = model1.transform(A_train)
doc_topic_data

array([[2.12675457e-05, 2.12675457e-05, 2.12675457e-05, ...,
        6.17106391e-01, 2.12675457e-05, 2.12675457e-05],
       [6.99202909e-06, 6.99202909e-06, 6.99202909e-06, ...,
        9.99867151e-01, 6.99202909e-06, 6.99202909e-06],
       [1.66611130e-05, 1.66611130e-05, 1.66611130e-05, ...,
        8.01512246e-01, 1.66611130e-05, 1.66611130e-05],
       ...,
       [5.25762355e-05, 5.25762355e-05, 5.25762355e-05, ...,
        9.99001052e-01, 5.25762355e-05, 5.25762355e-05],
       [4.54132607e-05, 4.54132607e-05, 4.54132607e-05, ...,
        4.54132607e-05, 4.54132607e-05, 4.54132607e-05],
       [8.40194925e-06, 8.40194925e-06, 8.40194925e-06, ...,
        9.99840363e-01, 8.40194925e-06, 8.40194925e-06]])

scikit-learnのLDAはどうやら正規化されていないため、正規化した上で、1つ目の文書がどのトピックから来ている単語が多いのかを見てみましょう。

In [16]:
normalize_doc_topic_data = \
 doc_topic_data / doc_topic_data.sum(axis=1, keepdims=True)

In [17]:
for topic_idx, prob in enumerate(normalize_doc_topic_data[0]):
    print('Topic #%d: probality: %f' % (topic_idx, prob))

Topic #0: probality: 0.000021
Topic #1: probality: 0.000021
Topic #2: probality: 0.000021
Topic #3: probality: 0.000021
Topic #4: probality: 0.000021
Topic #5: probality: 0.000021
Topic #6: probality: 0.000021
Topic #7: probality: 0.000021
Topic #8: probality: 0.382511
Topic #9: probality: 0.000021
Topic #10: probality: 0.000021
Topic #11: probality: 0.000021
Topic #12: probality: 0.000021
Topic #13: probality: 0.000021
Topic #14: probality: 0.000021
Topic #15: probality: 0.000021
Topic #16: probality: 0.000021
Topic #17: probality: 0.617106
Topic #18: probality: 0.000021
Topic #19: probality: 0.000021


In [18]:
loglikelihood = model1.score(A_test)
ppl = model1.perplexity(A_test)
print('対数尤度: ', loglikelihood)
print('Perplexity: ', ppl)

対数尤度:  -215572.08399223955
Perplexity:  4795.394248686762


テストデータに当てはめてみましょう。

In [19]:
test_doc_topic_data = model1.transform(A_test)
normalize_test_doc_topic_data = \
 test_doc_topic_data / test_doc_topic_data.sum(axis=1, keepdims=True)
for topic_idx, prob in enumerate(normalize_test_doc_topic_data[0]):
    print('Topic #%d: probality: %f' % (topic_idx, prob))

Topic #0: probality: 0.000014
Topic #1: probality: 0.000014
Topic #2: probality: 0.000014
Topic #3: probality: 0.582879
Topic #4: probality: 0.000014
Topic #5: probality: 0.000014
Topic #6: probality: 0.000014
Topic #7: probality: 0.000014
Topic #8: probality: 0.000014
Topic #9: probality: 0.000014
Topic #10: probality: 0.000014
Topic #11: probality: 0.000014
Topic #12: probality: 0.000014
Topic #13: probality: 0.000014
Topic #14: probality: 0.000014
Topic #15: probality: 0.000014
Topic #16: probality: 0.000014
Topic #17: probality: 0.416864
Topic #18: probality: 0.000014
Topic #19: probality: 0.000014


<h2> LDAを適用してみよう (ldaパッケージを使った場合)</h2> 

In [20]:
model2 = lda.LDA(n_topics=20, n_iter=1500, random_state=1, alpha=0.5, eta=0.5)

In [21]:
model2.fit(A_train)

INFO:lda:n_documents: 700
INFO:lda:vocab_size: 8709
INFO:lda:n_words: 60449
INFO:lda:n_topics: 20
INFO:lda:n_iter: 1500
/Users/takahi-i/.pyenv/versions/3.6.0/lib/python3.6/site-packages/lda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -690107
INFO:lda:<10> log likelihood: -537616
INFO:lda:<20> log likelihood: -508857
INFO:lda:<30> log likelihood: -504624
INFO:lda:<40> log likelihood: -502809
INFO:lda:<50> log likelihood: -502036
INFO:lda:<60> log likelihood: -501267
INFO:lda:<70> log likelihood: -501386
INFO:lda:<80> log likelihood: -501062
INFO:lda:<90> log likelihood: -500661
INFO:lda:<100> log likelihood: -500856
INFO:lda:<110> log likelihood: -500983
INFO:lda:<120> log likelihood: -500999
INFO:lda:<130> log likelihood: -500818
INFO:lda:<140> log likelihoo

In [22]:
topic_word = model2.topic_word_
n_top_words = 20
for topic_idx, topic in enumerate(topic_word):
    print('Topic #%d:' % topic_idx)
    print(' '.join([all_vocab_ar[i] for i in
                    topic.argsort()[:-n_top_words - 1:-1]]))
    print()

Topic #0:
格差 易 リンゴ 広瀬 お呼び 充当 域外 コラ 公明党 ブランディング 悲しみ ひよこ 歌 キッド 向き 意地 極寒 キャリア ラテックス 大手

Topic #1:
ワイヤ 召使い 圧勝 挽回 同右 岩波書店 やまもと 更新 ビル 大洲 ハンドル オネエ 同列 夏バテ シンサドン 元帥 創価大 夫 ふん ジャングル

Topic #2:
修士 博愛 ズブ 本来 指人形 模型 ボタン 激変 敏文 少子化 ぁぁあ 因子 ネットワ 受信 ロイヤルティ 晴れ ミニッツ 市原 ブリキ ひと安心

Topic #3:
悲劇 ワタ 昨日 おもてなし トモダチ 媛 来年度 椿 昨年 依存 択 橋本 気掛かり 内海 春愁 びねり 四条 本拠 やまもと 日本海

Topic #4:
ノイタミナ スランプ ランズ アダルト 泥棒 ドデカイ エデル 喜八 点字 シンバ エルメス 戦国 指令 典 のぶ 常時 健 ヘッドアップディスプレイ 外国 いい加減

Topic #5:
フルサイズ リッチ げた イニング 勇士 モコモコ ビジョン アレルバリア 先 南北 入念 とうもろこし ドリル デスク トゥットスポルト ポチ 将来 志村 アスパラガス ベネズエラ

Topic #6:
依存 区域 四条 津川 トヨ ポ アニメイト 流浪 やまもと ピグ ホイッスル 毒殺 喫煙 更新 大人 有名 おもてなし 柔らか ウルフ 昨日

Topic #7:
指数 別人 エマナ 切り出し 援助 実勢 ラジアルファン 挑 バルセロナキャンプ サイト 日活 ピンチ 態度 松吉 アイテム 朴 ハイカラ 厚手 役柄 気味

Topic #8:
宿命 入学 坂本 悲劇 ふみ クッション 樋口 ホアンキエム 昨日 キュンセリフ おしゃぶり ランキング シュシュ イングランド こまめ 勝人 ナイン 土日 マリリンジョイ テスト

Topic #9:
ピグ 昨日 ワタ 暁子 ウィル デスメタル 水平 ラベル いっしょ 制御 タブ 太もも 合 エルメス オフホワイト 大助 エルス アリストトリスト ルナ サンリオ

Topic #10:
宿命 塩尻 作家 今年度 タイヤ シリアス 流浪 あすか 比呂志 イタズラ ラベアン 根っこ 太平洋 ケバ 宮津 サガン 海流 メタル サザンセト 叫び

Topic #

今回も精度として対数尤度を見てみましょう

In [23]:
model2.loglikelihood()

-500732.333847963

文書 x トピック行列を見てみましょう。

In [24]:
doc_topic_data2 = model2.transform(A_train)
for topic_idx, prob in enumerate(doc_topic_data2[0]):
    print('Topic #%d: probality: %f' % (topic_idx, prob))

/Users/takahi-i/.pyenv/versions/3.6.0/lib/python3.6/site-packages/lda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):


Topic #0: probality: 0.033475
Topic #1: probality: 0.011446
Topic #2: probality: 0.005824
Topic #3: probality: 0.032966
Topic #4: probality: 0.005500
Topic #5: probality: 0.005773
Topic #6: probality: 0.021254
Topic #7: probality: 0.043399
Topic #8: probality: 0.009719
Topic #9: probality: 0.019212
Topic #10: probality: 0.091827
Topic #11: probality: 0.648025
Topic #12: probality: 0.010089
Topic #13: probality: 0.006127
Topic #14: probality: 0.005800
Topic #15: probality: 0.006053
Topic #16: probality: 0.017921
Topic #17: probality: 0.007934
Topic #18: probality: 0.013296
Topic #19: probality: 0.004362


<h2>（参考）ディリクレ分布の挙動</h2>

In [25]:
alpha = 0.1
K = 6
sampled_probs = np.random.dirichlet([alpha for i in range(K)])
for i, prob in enumerate(sampled_probs):
    print('サイコロ %d面  確率: %.2f'%(i+1, prob))

サイコロ 1面  確率: 0.40
サイコロ 2面  確率: 0.00
サイコロ 3面  確率: 0.60
サイコロ 4面  確率: 0.00
サイコロ 5面  確率: 0.00
サイコロ 6面  確率: 0.00
